In [2]:
pip install opencv-python

  Using cached opencv_python-4.10.0.84-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
Using cached opencv_python-4.10.0.84-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (62.5 MB)

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import cv2
import glob
import warnings
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
import itertools
from sklearn.metrics import confusion_matrix

2024-08-03 13:15:50.715573: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-03 13:15:51.905733: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-03 13:15:52.063682: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/abbywang/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/cv2/../../lib64:

In [4]:
# Ignore the warning
warnings.filterwarnings("ignore")
# Auto-reload extensions
%reload_ext autoreload
%autoreload 2
# Display Matplotlib plots inline 
%matplotlib inline

In [ ]:
##### Load the data from zipfile locally#####

# import zipfile

# zip_file_path = os.path.join('..', 'raw_data', 'archive.zip')
# extract_dir = os.path.join('..', 'raw_data', 'cnn_data')

# with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_dir)
    
# extracted_files = os.listdir(extract_dir)
# print("Extracted files:", extracted_files)    

# Data Exploration
1. remove the Grayscale images
2. resize images
3. normalization
4. argumentation

## Remove Grayscale images

In [6]:
styles = [
    'Achaemenid architecture',
    'American craftsman style',
    'American Foursquare architecture',
    'Ancient Egyptian architecture',
    'Art Deco architecture',
    'Art Nouveau architecture',
    'Baroque architecture',
    'Bauhaus architecture',
    'Beaux-Arts architecture',
    'Byzantine architecture',
    'Chicago school architecture',
    'Colonial architecture',
    'Deconstructivism',
    'Edwardian architecture',
    'Georgian architecture',
    'Gothic architecture',
    'Greek Revival architecture',
    'International style',
    'Novelty architecture',
    'Palladian architecture',
    'Postmodern architecture',
    'Queen Anne architecture',
    'Romanesque architecture',
    'Russian Revival architecture',
    'Tudor Revival architecture'
] 

In [25]:
from PIL import Image

base_folder_path = '../raw_data/cnn_data/architectural-styles-dataset'

def black_white(image_path):
    image = Image.open(image_path).convert('RGB')
    image_array = np.array(image)
    
    if np.all(image_array[:, :, 0] == image_array[:, :, 1]) and  np.all(image_array[:, :, 1]== image_array[:, :, 2]):
        return True 
    else:
        return False 
        

def process_images_in_style(style_name):
    folder_path = os.path.join(base_folder_path, style_name)
    deleted_count = 0
    if not os.path.exists(folder_path):
        print(f"Folder not found: {folder_path}")
        return
    
    for file_name in os.listdir(folder_path):
        image_path = os.path.join(folder_path, file_name)
        if black_white(image_path):
            os.remove(image_path)
            deleted_count += 1
#             print(f"Grayscale image found: {file_name} in {style_name}")

    print(f"Deleted {deleted_count} files in the {style_name}") 
                    

In [26]:
# Process images for each folder
for style in styles:
    print(f"Processing images for style: {style}")
    process_images_in_style(style)   

Processing images for style: Achaemenid architecture
Deleted 2 files in the Achaemenid architecture
Processing images for style: American craftsman style
Deleted 0 files in the American craftsman style
Processing images for style: American Foursquare architecture
Deleted 3 files in the American Foursquare architecture
Processing images for style: Ancient Egyptian architecture
Deleted 0 files in the Ancient Egyptian architecture
Processing images for style: Art Deco architecture
Deleted 10 files in the Art Deco architecture
Processing images for style: Art Nouveau architecture
Deleted 8 files in the Art Nouveau architecture
Processing images for style: Baroque architecture
Deleted 2 files in the Baroque architecture
Processing images for style: Bauhaus architecture
Deleted 27 files in the Bauhaus architecture
Processing images for style: Beaux-Arts architecture
Deleted 7 files in the Beaux-Arts architecture
Processing images for style: Byzantine architecture
Deleted 4 files in the Byzan

## Split the dataset into Train/Val/Test

## Resize and Normalize the Images

In [ ]:
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.layers import Rescaling


train_ds = image_dataset_from_directory(
  train_data_dir,
  labels = "inferred",
  label_mode = "binary",
  seed=123,
  image_size=(150, 150),
  batch_size=batch_size)

# We define a second one for the test data

val_ds = image_dataset_from_directory(
  test_data_dir,
  labels = "inferred",
  label_mode = "binary",
  seed=123,
  image_size=(150, 150),
  batch_size=batch_size)